## Reidintification Notebook
### Deep Learning project - Task #1
*Sebastiano Chiari - Francesco Ferrini - Wamiq Raza*

### Setup

Import section

In [1]:
import pandas as pd  
import numpy as np   
import csv 

import torch
from torch import nn, optim
import torchvision
from torchvision import transforms, datasets, models, utils
from torch.utils.data import Dataset, DataLoader 
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
from skimage import io, transform
from sklearn.metrics import pairwise_distances
from torch.optim import lr_scheduler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
import torchvision.transforms as T

import os
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
import random
from torch.utils.tensorboard import SummaryWriter

Global variables definition

In [2]:
DATASET_PATH = '/content/dataset/'
MODEL_PATH = '/content/reid_model.pth'

TRAIN_FOLDER_PATH = DATASET_PATH + 'train'
ANNOTATION_CSV_PATH = DATASET_PATH + 'annotations_train.csv'

QUERY_CSV_PATH = DATASET_PATH + 'queries.csv'
QUERY_FOLDER_PATH = DATASET_PATH + 'queries/'
TEST_CSV_PATH = DATASET_PATH + 'tests.csv'
TEST_FOLDER_PATH = DATASET_PATH + 'test/'

In [3]:
TRAIN_PERCENTAGE = 80
BATCH_SIZE = 64
EPOCHS = 100

Device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
device

## Dataset

Import the dataset and extract the zip file

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
!unzip "/content/drive/MyDrive/dataset.zip" -d dataset

### Early Stopping

patience of 7

In [7]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path=MODEL_PATH, trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}, {type(self.counter)}, {type(self.patience)}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model, self.path)
        self.val_loss_min = val_loss


## mAP computation

In [8]:
from typing import Dict, Set, List


class Evaluator:

    @staticmethod
    def evaluate_map(predictions: Dict[str, List], ground_truth: Dict[str, Set]):
        '''
        Computes the mAP (https://jonathan-hui.medium.com/map-mean-average-precision-for-object-detection-45c121a31173) of the predictions with respect to the given ground truth
        In person reidentification mAP refers to the mean of the AP over all queries.
        The AP for a query is the area under the precision-recall curve obtained from the list of predictions considering the
        ground truth elements as positives and the other ones as negatives

        :param predictions: dictionary from query filename to list of test image filenames associated with the query ordered
                            from the most to the least confident prediction.
                            Represents the predictions to be evaluated.
        :param ground_truth: dictionary from query filename to set of test image filenames associated with the query
                             Represents the ground truth on which to evaluate predictions.

        :return:
        '''

        m_ap = 0.0
        for current_ground_truth_query, current_ground_truth_query_set in ground_truth.items():

            # No predictions were performed for the current query, AP = 0
            if not current_ground_truth_query in predictions:
                continue

            current_ap = 0.0  # The area under the curve for the current sample
            current_predictions_list = predictions[current_ground_truth_query]

            # Recall increments of this quantity each time a new correct prediction is encountered in the prediction list
            delta_recall = 1.0 / len(current_ground_truth_query_set)

            # Goes through the list of predictions
            encountered_positives = 0
            for idx, current_prediction in enumerate(current_predictions_list):
                # Each time a positive is encountered, compute the current precition and the area under the curve
                # since the last positive
                if current_prediction in current_ground_truth_query_set:
                    encountered_positives += 1
                    current_precision = encountered_positives / (idx + 1)
                    current_ap += current_precision * delta_recall

            m_ap += current_ap

        # Compute mean over all queries
        m_ap /= len(ground_truth)

        return m_ap

Preprocessing (create one new csv file without the attributes)

In [9]:
def new_annotation(tr_path, ann_path):
  # read csv
  ids = pd.read_csv(ann_path)
  train_imgs = os.listdir(tr_path)

  # crate a dataframe
  train_imgs_data = []
  for s in train_imgs:
    train_imgs_data.append([s, int(s.split('_')[0])])
  #print(train_imgs_data)

  train_imgs_df = pd.DataFrame(train_imgs_data, columns=['path', 'id'])

  train_imgs_df=train_imgs_df.sort_values("id")
  train_imgs_df.reset_index(inplace=True)
  train_imgs_df.drop("index",axis=1,inplace=True)
  
  # Encode target labels with value between 0 and n_classes-1.
  label_encoder = preprocessing.LabelEncoder()
  train_imgs_df['id']= label_encoder.fit_transform(train_imgs_df['id'])
  annotation_reid_path = DATASET_PATH + 'annotations_reid.csv'
  train_imgs_df.to_csv(annotation_reid_path)
  train_imgs_df.head()


  return annotation_reid_path

annotation_reid_path= new_annotation(TRAIN_FOLDER_PATH, ANNOTATION_CSV_PATH)

Train dataset class

In [10]:
from torchvision.io import read_image
class custom_dataset(Dataset):
    def __init__(self, csv_file, root_dir, transform):
        self.annotations = pd.read_csv(csv_file,index_col=[0])
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir,str(self.annotations.iloc[index, 0]))
        image = read_image(img_path)
        image=image.squeeze().permute(1,2,0)
        label_id=np.array([self.annotations.iloc[index,1]]).astype('int')

        anchor_img = image
        anchor_label = int(label_id)   

        positive_list = self.annotations[self.annotations["id"]==anchor_label].index
        positive_item = random.choice(positive_list)
        positive_img = self.annotations.path[positive_item]
        positive_img_path=os.path.join(self.root_dir,positive_img)
        positive_img=read_image(positive_img_path)


        negative_list = self.annotations[self.annotations["id"]!=anchor_label].index
        negative_item = random.choice(negative_list)
        negative_img = self.annotations.path[negative_item]
        negative_img_path=os.path.join(self.root_dir,negative_img)
        negative_img=read_image(negative_img_path)
        sample={"anchor_image":np.uint8(image),"img_path":img_path,"positive_image":positive_img,"negative_image":negative_img,'label_id': label_id}

        if self.transform:
            sample["anchor_image"] = self.transform(sample["anchor_image"])
            sample["positive_image"] = self.transform(sample["positive_image"])
            sample["negative_image"] = self.transform(sample["negative_image"])
       
        return (sample)



Test dataset class

In [11]:
class custom_dataset_test(Dataset):
    def __init__(self, csv_file, root_dir, transform):
        self.annotations = pd.read_csv(csv_file,index_col=[0])
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir,str(self.annotations.iloc[index, 0]))
        image = read_image(img_path)
        image=image.squeeze().permute(1,2,0)

        sample={"image":np.uint8(image),"image_path":img_path}

        if self.transform:
            sample["image"] = self.transform(sample["image"])
        return (sample)

Data augmentation



In [12]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

tfms = T.Compose([
    T.ToPILImage(),
    T.RandomCrop((128, 64), padding=8, padding_mode='reflect'),
    T.RandomHorizontalFlip(p=0.5), 
    T.RandomRotation(10),
    T.ToTensor(), 
    T.Normalize(*imagenet_stats,inplace=True), 
    T.RandomErasing(p=0.5, inplace=True)
])

valid_tfms = T.Compose([
    T.ToTensor(), 
    T.Normalize(*imagenet_stats)
])

## Dataloaders



In [13]:
def get_data(batch_size):
  # Load data
  dataset = custom_dataset(
      csv_file=annotation_reid_path,
      root_dir=TRAIN_FOLDER_PATH,
      transform=tfms)
      
  tr_size = int(len(dataset) * TRAIN_PERCENTAGE/100)
  val_size = int(len(dataset)) - tr_size
  train, valid=torch.utils.data.random_split(dataset, [tr_size,val_size])

  # Initialize dataloaders
  train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
  valid_loader = torch.utils.data.DataLoader(valid, batch_size, shuffle=True)
  
  return train_loader, valid_loader

train_loader, valid_loader = get_data(batch_size=BATCH_SIZE)

In [14]:
def get_data_test():
  query_dataset = custom_dataset_test(
      csv_file=QUERY_CSV_PATH,
      root_dir=QUERY_FOLDER_PATH,
      transform=valid_tfms)
      
  query_loader = torch.utils.data.DataLoader(query_dataset, 32, shuffle=True)

  test_dataset = custom_dataset_test(
        csv_file=TEST_CSV_PATH,
        root_dir=TEST_FOLDER_PATH,
        transform=valid_tfms)
        
  test_loader = torch.utils.data.DataLoader(test_dataset, 32, shuffle=True)

  return query_loader, test_loader


## Network definition

ResNet50 pretrained

In [ ]:
class ResNet50(nn.Module):
  def __init__(self, **kwargs):
    super(ResNet50, self).__init__()
    # returns a model consisting of all layers of resnet50 but the last one (a fully connected layer), with fixed parameters
    resnet50 = models.resnet50(pretrained=True)
    modules=list(resnet50.children())[:-1]
    self.base=nn.Sequential(*modules)

  def forward(self, x):
    x = self.base(x)
    y = x.view(x.size(0), -1)

    return y

model = ResNet50().to(device)

## Loss / cost function

In [16]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    
    def forward(self, anchor: torch.Tensor, positive: torch.Tensor, negative: torch.Tensor) -> torch.Tensor:
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)

        return losses.mean()


def triplet_cost_function():
  cost_function = torch.jit.script(TripletLoss())
  return cost_function

## Optimizer

In [17]:
def get_optimizer(model, lr, wd, momentum):
  
  #optimizer=torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999),weight_decay=wd, eps=1e-08, amsgrad=False)
  optimizer=torch.optim.RMSprop(model.parameters(), lr = lr, alpha = 0.9)
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
  #optimizer=torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum,weight_decay=wd)
  return optimizer


## Train step

In [18]:
def train_model(model,train_loader,criterion,optimizer):
  train_loss = 0.0
  samples= 0.0
  cumulative_train_loss = 0.0
  running_loss=[]

  model.train()
  for batch_idx, sample in enumerate(tqdm(train_loader, desc="Training", leave=False)):

      # Load data into GPU
      anchor_img = sample['anchor_image'].to(device)
      positive_img = sample['positive_image'].to(device)
      negative_img= sample['negative_image'].to(device)
      anchor_label = sample['label_id'].to(device)

      labels=anchor_label.type(torch.LongTensor)
                                                                                                 
      # Reset the gradients
      optimizer.zero_grad()

      # Forward pass
      anchor_out = model(anchor_img)
      positive_out = model(positive_img)
      negative_out = model(negative_img)
                  
      # Apply the loss
      loss = criterion(anchor_out, positive_out, negative_out)

      # Backward pass
      loss.backward()

      # Update parameters
      optimizer.step()
      cumulative_train_loss += loss.item()

      running_loss.append(loss.cpu().detach().numpy())

      train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))

  return np.mean(running_loss)

## Validation step

In [19]:
def valid_model(model,valid_loader ,criterion):
  valid_loss = 0.0
  best=np.inf
  samples= 0.0
  cumulative_valid_loss = 0.0
  valid_results = []
  labels_for_prediction = []
  running_loss=[]

  model.eval()
  with torch.no_grad():
    for batch_idx, sample in enumerate(tqdm(valid_loader, desc="Validation", leave=False)):

        # Load data into GPU
        anchor_img = sample['anchor_image'].to(device)
        positive_img = sample['positive_image'].to(device)
        negative_img= sample['negative_image'].to(device)
        anchor_label = sample['label_id'].to(device)
        
        labels = anchor_label.type(torch.LongTensor)
        
        valid_results.append(model(anchor_img).cpu().numpy())
        labels_for_prediction.append(anchor_label)
                                                                                                          
        # Forward pass
        anchor_out = model(anchor_img)
        positive_out = model(positive_img)
        negative_out = model(negative_img)
                    
        # Apply the loss
        loss = criterion(anchor_out, positive_out, negative_out)

        cumulative_valid_loss += loss.item()

        running_loss.append(loss.cpu().detach().numpy())

        valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
    

  return np.mean(running_loss)

## Main

In [20]:
def log_values(writer, step, loss, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)

In [21]:
def main(batch_size=BATCH_SIZE, 
         device='cuda:0', 
         learning_rate=0.001, 
         weight_decay=0.01, 
         momentum=0.9, 
         epochs=3, 
         train_loader=train_loader, 
         valid_loader=valid_loader, 
         checkpoint_path=MODEL_PATH):
  
  # Creates a logger for the experiment
  writer = SummaryWriter(log_dir="runs/loss")

  early_stopping = EarlyStopping(verbose=True)
  
  # Instantiates the model
  net=ResNet50().to(device)

  # Instantiates the optimizer
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
  
  # Instantiates the cost function
  cost_function = triplet_cost_function()

  print('Before training:')
  train_loss= train_model(net,train_loader ,cost_function, optimizer)
  valid_loss= valid_model(net,valid_loader,cost_function)
  

  print('\t Training loss {:.5f}'.format(train_loss))
  print('\t Valid loss {:.5f}'.format(valid_loss))
  print('-----------------------------------------------------')
    
  # Add values to plots
  writer.add_scalar('Loss/train_loss', train_loss, 0)
  writer.add_scalar('Loss/valid_loss', valid_loss, 0)

  for e in tqdm(range(epochs), desc="Epochs"):
    train_loss= train_model(net,train_loader ,cost_function, optimizer)
    valid_loss= valid_model(net,valid_loader,cost_function)
      
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}'.format(train_loss))
    print('\t Valid loss {:.5f}'.format(valid_loss))
    
     # log to tensorboard
    log_values(writer, e, train_loss,"Train")
    log_values(writer, e, valid_loss, "Validation")

    early_stopping(valid_loss, net)

    if early_stopping.early_stop:
      print("Early stopping has occurred. Reverting to latest save model")
      break
    
  # Closes the logger
  writer.close()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [ ]:
main()

#mAP Calculation

Load the model

In [24]:
model = torch.load(MODEL_PATH)

Validate model

In [25]:
def validate_model(model, loader):
  model.eval()

  results = []
  img_names = []
  labels = []
  with torch.no_grad():
      for step, sample in enumerate(tqdm(loader)):
          anchor_img,img_path = sample['anchor_image'].to(device),sample['img_path']
          anchor_label = sample['label_id']
          results.append(model(anchor_img).cpu().numpy())
          img_names.append(img_path)
          labels.append(anchor_label)

  results = np.concatenate(results)
  img_names = np.concatenate(img_names)
  labels = np.concatenate(labels)
  print(img_names.shape)
  print(results.shape)
  print(labels.shape)

  return results, img_names, labels

In [ ]:
val_results, val_img_names, val_labels = validate_model(model, valid_loader)

## Predictions and groundtruth


In [31]:
def predict(validation_results, validation_img_names):
  '''
    Calculating predictions on the validation set
  '''
  validation_results_df = pd.DataFrame(validation_results)
  validation_results_df['img_path'] = validation_img_names
  ids_ = []
  for k in validation_results_df['img_path']:
    ids_.append(k.split('/')[4].split('_')[0])
  validation_results_df['person_id'] = ids_ # val_results is a dataframe where each row contains the image name, the path and all the 20148 features

  cosine_df=pd.DataFrame(cosine_similarity(validation_results_df.iloc[:,:-2], validation_results_df.iloc[:,:-2], dense_output=True))
  cosine_df=cosine_df.set_index(validation_results_df.img_path)
  cosine_df.columns=validation_results_df.img_path # cosine_df is a dataframe thatcontains in each cell the cosine similarity btw image in row with image in column
  predictions = {}
  threshold = 0.99

  for i in tqdm(range(len(cosine_df))):
    tmp_df = pd.DataFrame(cosine_df.iloc[i].sort_values(ascending=False))
    id = (tmp_df.columns[0]).split('/')[4].split('_')[0]
    images_list = []
    for index, row in tmp_df.iterrows():
      value = row[0]
      if(value < 1.0 and value >= threshold):
        images_list.append(index)
      if(value < threshold):
        break
    predictions[id] = images_list
  '''
    Calculating the groundtruth in order to compute the mAP
  '''
  gt = {}
  for id in ids_:
    img_list = []
    for img_name in validation_img_names:
      if id == img_name.split('/')[4].split('_')[0]:
        img_list.append(img_name)
    gt[id] = set(img_list)
  
  return predictions, gt
  

In [ ]:
predictions_dict, groundtruth_dict = predict(val_results, val_img_names)

##mAP result

In [ ]:
Evaluator.evaluate_map(predictions_dict, groundtruth_dict)

# Submission

Load the model

In [34]:
model = torch.load(MODEL_PATH)

Test the model

In [35]:
def test_model(model, loader):
  model.eval()

  results = []
  img_names = []
  with torch.no_grad():
      for step, sample in enumerate(tqdm(loader)):
          anchor_img,img_path = sample['image'].to(device),sample['image_path']
          results.append(model(anchor_img).cpu().numpy())
          img_names.append(img_path)

  results = np.concatenate(results)
  img_names = np.concatenate(img_names)
  print(img_names.shape)
  print(results.shape)

  return results, img_names

Create a sumbission dictionary to be converted in a txt

In [42]:
def submit():
  # Query csv
  query_imgs = os.listdir(QUERY_FOLDER_PATH)
  query_df = pd.DataFrame(query_imgs, columns=['img_names'])
  query_df.to_csv(QUERY_CSV_PATH)
  # Test csv
  test_imgs = os.listdir(TEST_FOLDER_PATH)
  test_df = pd.DataFrame(test_imgs, columns=['img_names'])
  test_df.to_csv(TEST_CSV_PATH)

  # Get dataloaders for query and test
  query_loader, test_loader = get_data_test()

  # Make predictions on query and test
  q_results, queries_img_names = test_model(model, query_loader)
  t_results, test_img_names = test_model(model, test_loader)

  # Create a dataframe for both
  queries_results=pd.DataFrame(q_results)
  queries_results["img_name"]=queries_img_names
  test_results=pd.DataFrame(t_results)
  test_results["img_name"]=test_img_names

  # Cosine similarity dataframe
  submission_cos_sim=pd.DataFrame(cosine_similarity(queries_results.iloc[:,:-1], test_results.iloc[:,:-1])) # apply cosine similarity to the 2 vectors of features but the last column
  submission_cos_sim=submission_cos_sim.set_index(queries_results.img_name)
  submission_cos_sim.columns=test_results.img_name

  # Make final predictions for sumbission
  submissions = {}
  threshold = 0.99
  for i in tqdm(range(len(submission_cos_sim))):
    tmp_df = pd.DataFrame(submission_cos_sim.iloc[i].sort_values(ascending=False))
    id = (tmp_df.columns[0]).split('/')[4].split('_')[0]
    # images list
    images_list = []
    for index, row in tmp_df.iterrows():
      value = row[0]
      if(value < 1.0 and value >= threshold):
        images_list.append(index.split('/')[4].split('_')[0])
      if(value < threshold):
        break
    submissions[id] = images_list

  return submissions

In [ ]:
submissions_dict = submit()

Convert dictionary to txt and save it

In [46]:
l = []
for key in submissions_dict.keys():
  string = ''
  string = string + key + ': '
  comma = 0
  for elm in submissions_dict[key]:
    if (comma == 0):
      string = string + elm
      comma = 1
    if(comma != 0):
      string = string + ', ' + elm
  l.append(string)
f = open('/content/dataset/reid_test.txt', 'w')
for st in l:
  f.write(st)
  f.write('\n')
f.close()